In [1]:
import numpy as np
import detectron2
from detectron2.modeling import build_model
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
import glob
import pickle
import cv2
import os
import json
from matplotlib import pyplot as plt
from jupyterthemes import jtplot
jtplot.style(ticks=True, grid=False, figsize=(20, 10))

In [2]:

cfg = get_cfg()
cfg.merge_from_file(r"output/config.yaml")
pred = DefaultPredictor(cfg)
imgpath = glob.glob(r"datasets/cityscapes/leftImg8bit/val/*/*.png")
gtpath = glob.glob(r"datasets/cityscapes/gtFine/val/*/*.json")
imgpath.sort()
gtpath.sort()
print(imgpath[0], gtpath[0])

datasets/cityscapes/leftImg8bit/val/frankfurt/frankfurt_000000_000294_leftImg8bit.png datasets/cityscapes/gtFine/val/frankfurt/frankfurt_000000_000294_gtFine_polygons.json


In [21]:
# clear proposals folder beforehand
for i in range(500):
    if os.path.exists(r"proposals/img" + str(i) + r".pickle"):
        os.remove(r"proposals/img" + str(i) + r".pickle")

for i in range(500):
    if (i % 100 == 99):
        print("progress: {} / 500".format(i))
    img = cv2.imread(imgpath[i])
    output = pred(img)
    

progress: 99 / 500
progress: 199 / 500
progress: 299 / 500
progress: 399 / 500
progress: 499 / 500


In [27]:
def print_rect(xy, w, h, color):
    plt.gca().add_patch(plt.Rectangle(xy=xy,
    width=w, 
    height=h,
    fill=False, linewidth=1, edgecolor=color))

def area(box):
    return max(0, (box[2] - box[0])) * max(0, (box[3] - box[1]))
    
def iou(box1, box2):
    U = [max(box1[0], box2[0]), max(box1[1], box2[1]), min(box1[2], box2[2]), min(box1[3], box2[3])]
    return area(U) / (area(box1) + area(box2) - area(U) + 1e-6)
    
def pairwise_iou(boxs1, boxs2):
    out = np.zeros((len(boxs1), len(boxs2)))
    assert(len(boxs1) > 0)
    assert(len(boxs2) > 0)
    for i, box1 in enumerate(boxs1):
        for j, box2 in enumerate(boxs2):
            out[i, j] = iou(box1, box2)
    return out

selection = ['person', 'rider', 'car', 'truck', 'bus', 'caravan', 'trailer', 'train', 'motorcycle', 'bicycle']
threshold = 0.7
mean_recall = 0
mean_accuracy = 0
empty_count = 0

# read pickle files
for i in range(0, 500):
    proposals = None
    with open("proposals/img" + str(i) + ".pickle", "rb") as file:
        proposals = pickle.load(file)
    proposals = proposals[0].proposal_boxes.tensor.cpu().numpy()
#     img = cv2.imread(imgpath[i])
#     b,g,r=cv2.split(img)
#     img = cv2.merge([r,g,b])
#     plt.imshow(img)
#     for box in proposals:
#         print_rect((box[0], box[1]), box[2] - box[0], box[3] - box[1], 'b')
        
    # gt
    with open(gtpath[i], 'r') as file:
        data = file.read()
    gts = json.loads(data)["objects"]
    gtboxes = []# N * 4
    for gt in gts:
        if not gt['label'] in selection:
            continue
        polygon = gt['polygon']
        gtboxes.append([9999, 9999, 0, 0])
        for point in polygon:
            gtboxes[-1][0] = min(gtboxes[-1][0], point[0])
            gtboxes[-1][1] = min(gtboxes[-1][1], point[1])
            gtboxes[-1][2] = max(gtboxes[-1][2], point[0])
            gtboxes[-1][3] = max(gtboxes[-1][3], point[1])
#     for box in gtboxes:
#         print_rect((box[0], box[1]), box[2] - box[0], box[3] - box[1], 'r')
    if len(proposals) == 0 or len(gtboxes) == 0:
        empty_count += 1
        continue
    ious = pairwise_iou(proposals, gtboxes)
    recall = np.sum(np.max(ious, axis=0, keepdims=True) > threshold) / len(gtboxes)
    accuracy = np.sum(np.max(ious, axis=1, keepdims=True) > threshold) / len(proposals)
    mean_recall += recall
    mean_accuracy += accuracy
    print("i =", i)
    print("accuracy =", accuracy)
    print("recall =", recall)
    
mean_recall /= 500 - empty_count
mean_accuracy /= 500 - empty_count
print("mean_recall =", mean_recall, "mean_accuracy =", mean_accuracy)

i = 0
accuracy = 0.024
recall = 1.0
i = 1
accuracy = 0.057
recall = 0.9047619047619048
i = 2
accuracy = 0.027
recall = 1.0
i = 3
accuracy = 0.056
recall = 0.6875
i = 4
accuracy = 0.061
recall = 0.92
i = 5
accuracy = 0.055
recall = 0.9047619047619048
i = 6
accuracy = 0.044
recall = 0.8235294117647058
i = 7
accuracy = 0.05
recall = 0.85
i = 8
accuracy = 0.057
recall = 0.8888888888888888
i = 9
accuracy = 0.015
recall = 0.7142857142857143
i = 10
accuracy = 0.025
recall = 0.9
i = 11
accuracy = 0.029
recall = 0.8
i = 12
accuracy = 0.044
recall = 0.8571428571428571
i = 13
accuracy = 0.031
recall = 0.9
i = 14
accuracy = 0.017
recall = 0.8571428571428571
i = 15
accuracy = 0.039
recall = 0.7777777777777778
i = 16
accuracy = 0.038
recall = 0.8125
i = 17
accuracy = 0.037
recall = 0.9230769230769231
i = 18
accuracy = 0.04
recall = 0.8333333333333334
i = 19
accuracy = 0.035
recall = 1.0
i = 20
accuracy = 0.039
recall = 1.0
i = 21
accuracy = 0.013
recall = 1.0
i = 22
accuracy = 0.012
recall = 1.0
i =